# Groupby

* split -> apply -> combine
* 위 과정을 거쳐 연산함

In [1]:
import numpy as np
import pandas as pd
from pandas import Series, DataFrame

In [2]:
# data from: 
ipl_data = {'Team': ['Riders', 'Riders', 'Devils', 'Devils', 'Kings',
         'kings', 'Kings', 'Kings', 'Riders', 'Royals', 'Royals', 'Riders'],
         'Rank': [1, 2, 2, 3, 3,4 ,1 ,1,2 , 4,1,2],
         'Year': [2014,2015,2014,2015,2014,2015,2016,2017,2016,2014,2015,2017],
         'Points':[876,789,863,673,741,812,756,788,694,701,804,690]}

df = pd.DataFrame(ipl_data)
df

,Team,Rank,Year,Points
0,Riders,1,2014,876
1,Riders,2,2015,789
2,Devils,2,2014,863
3,Devils,3,2015,673
4,Kings,3,2014,741
5,kings,4,2015,812
6,Kings,1,2016,756
7,Kings,1,2017,788
8,Riders,2,2016,694
9,Royals,4,2014,701


## Hierarchical index

* 한 개이상의 column을 묶을 수 있음 

In [5]:
h_index = df.groupby(["Team", "Year"])["Points"].sum()
h_index

Team    Year
Devils  2014    863
        2015    673
Kings   2014    741
        2016    756
        2017    788
Riders  2014    876
        2015    789
        2016    694
        2017    690
Royals  2014    701
        2015    804
kings   2015    812
Name: Points, dtype: int64

In [6]:
h_index.index

MultiIndex(levels=[['Devils', 'Kings', 'Riders', 'Royals', 'kings'], [2014, 2015, 2016, 2017]],
           labels=[[0, 0, 1, 1, 1, 2, 2, 2, 2, 3, 3, 4], [0, 1, 0, 2, 3, 0, 1, 2, 3, 0, 1, 1]],
           names=['Team', 'Year'])

In [7]:
h_index["Devils":"Riders"]

Team    Year
Devils  2014    863
        2015    673
Kings   2014    741
        2016    756
        2017    788
Riders  2014    876
        2015    789
        2016    694
        2017    690
Name: Points, dtype: int64

In [8]:
h_index.unstack()

Year,2014,2015,2016,2017
Team,,,,
Devils,863.0,673.0,NaN,NaN
Kings,741.0,NaN,756.0,788.0
Riders,876.0,789.0,694.0,690.0
Royals,701.0,804.0,NaN,NaN
kings,NaN,812.0,NaN,NaN


## Hierarchical index - swaplevel

* index lavel을 변경할 수 있음

In [9]:
h_index.swaplevel()

Year  Team  
2014  Devils    863
2015  Devils    673
2014  Kings     741
2016  Kings     756
2017  Kings     788
2014  Riders    876
2015  Riders    789
2016  Riders    694
2017  Riders    690
2014  Royals    701
2015  Royals    804
      kings     812
Name: Points, dtype: int64

In [11]:
h_index.sort_values(ascending=False)

Team    Year
Riders  2014    876
Devils  2014    863
kings   2015    812
Royals  2015    804
Riders  2015    789
Kings   2017    788
        2016    756
        2014    741
Royals  2014    701
Riders  2016    694
        2017    690
Devils  2015    673
Name: Points, dtype: int64

In [14]:
h_index.swaplevel().sort_index(level=0)

Year  Team  
2014  Devils    863
      Kings     741
      Riders    876
      Royals    701
2015  Devils    673
      Riders    789
      Royals    804
      kings     812
2016  Kings     756
      Riders    694
2017  Kings     788
      Riders    690
Name: Points, dtype: int64

In [16]:
h_index.sum(level="Team")

Team
Devils    1536
Kings     2285
Riders    3049
Royals    1505
kings      812
Name: Points, dtype: int64

## Groupby - grouped

* groupby에 의해 split된 상태를 추출 가능함

In [17]:
grouped = df.groupby("Team")
for name, group in grouped:
    print(name)
    print(group)

Devils
     Team  Rank  Year  Points
2  Devils     2  2014     863
3  Devils     3  2015     673
Kings
    Team  Rank  Year  Points
4  Kings     3  2014     741
6  Kings     1  2016     756
7  Kings     1  2017     788
Riders
      Team  Rank  Year  Points
0   Riders     1  2014     876
1   Riders     2  2015     789
8   Riders     2  2016     694
11  Riders     2  2017     690
Royals
      Team  Rank  Year  Points
9   Royals     4  2014     701
10  Royals     1  2015     804
kings
    Team  Rank  Year  Points
5  kings     4  2015     812


In [18]:
grouped.get_group("Riders")

,Team,Rank,Year,Points
0,Riders,1,2014,876
1,Riders,2,2015,789
8,Riders,2,2016,694
11,Riders,2,2017,690


추출된 group 정보에는 세 가지 유형의 apply가 가능함

* Aggregation: 요약된 통계정보를 추출해 줌
* Transformation : 해당 정보를 변환해 줌
* Filtration: 특정 정보를 제거하여 보여주는 필터링 기능

In [19]:
# Aggregation
grouped.agg(sum)

,Rank,Year,Points
Team,,,
Devils,5,4029,1536
Kings,5,6047,2285
Riders,7,8062,3049
Royals,5,4029,1505
kings,4,2015,812


In [20]:
grouped.agg(np.mean)

,Rank,Year,Points
Team,,,
Devils,2.500000,2014.500000,768.000000
Kings,1.666667,2015.666667,761.666667
Riders,1.750000,2015.500000,762.250000
Royals,2.500000,2014.500000,752.500000
kings,4.000000,2015.000000,812.000000


In [21]:
grouped['Points'].agg([np.sum, np.mean, np.std])

,sum,mean,std
Team,,,
Devils,1536,768.000000,134.350288
Kings,2285,761.666667,24.006943
Riders,3049,762.250000,88.567771
Royals,1505,752.500000,72.831998
kings,812,812.000000,NaN


In [23]:
grouped.describe().T

Team               Devils        Kings       Riders       Royals   kings
Points count     2.000000     3.000000     4.000000     2.000000     1.0
       mean    768.000000   761.666667   762.250000   752.500000   812.0
       std     134.350288    24.006943    88.567771    72.831998     NaN
       min     673.000000   741.000000   690.000000   701.000000   812.0
       25%     720.500000   748.500000   693.000000   726.750000   812.0
       50%     768.000000   756.000000   741.500000   752.500000   812.0
       75%     815.500000   772.000000   810.750000   778.250000   812.0
       max     863.000000   788.000000   876.000000   804.000000   812.0
Rank   count     2.000000     3.000000     4.000000     2.000000     1.0
       mean      2.500000     1.666667     1.750000     2.500000     4.0
       std       0.707107     1.154701     0.500000     2.121320     NaN
       min       2.000000     1.000000     1.000000     1.000000     4.0
       25%       2.250000     1.000000     1.750000     1.750000     4.0
       50%       2.500000     1.000000     2.000000     2.500000     4.0
       75%       2.750000     2.000000     2.000000     3.250000     4.0
       max       3.000000     3.000000     2.000000     4.000000     4.0
Year   count     2.000000     3.000000     4.000000     2.000000     1.0
       mean   2014.500000  2015.666667  2015.500000  2014.500000  2015.0
       std       0.707107     1.527525     1.290994     0.707107     NaN
       min    2014.000000  2014.000000  2014.000000  2014.000000  2015.0
       25%    2014.250000  2015.000000  2014.750000  2014.250000  2015.0
       50%    2014.500000  2016.000000  2015.500000  2014.500000  2015.0
       75%    2014.750000  2016.500000  2016.250000  2014.750000  2015.0
       max    2015.000000  2017.000000  2017.000000  2015.000000  2015.0

In [33]:
# transformation
score = lambda x: x.max()
print(df)
grouped.transform(score)

      Team  Rank  Year  Points
0   Riders     1  2014     876
1   Riders     2  2015     789
2   Devils     2  2014     863
3   Devils     3  2015     673
4    Kings     3  2014     741
5    kings     4  2015     812
6    Kings     1  2016     756
7    Kings     1  2017     788
8   Riders     2  2016     694
9   Royals     4  2014     701
10  Royals     1  2015     804
11  Riders     2  2017     690


,Rank,Year,Points
0,2,2017,876
1,2,2017,876
2,3,2015,863
3,3,2015,863
4,3,2017,788
5,4,2015,812
6,3,2017,788
7,3,2017,788
8,2,2017,876
9,4,2015,804


## Groupby - Filter

* 특정 조건으로 데이터를 검색할 때 사용

In [37]:
df.groupby('Team').filter(lambda x: len(x) >= 3) # grouped된 dataframe 개수

,Team,Rank,Year,Points
0,Riders,1,2014,876
1,Riders,2,2015,789
4,Kings,3,2014,741
6,Kings,1,2016,756
7,Kings,1,2017,788
8,Riders,2,2016,694
11,Riders,2,2017,690
